In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from retrying import retry
import time

start_time = time.time()

baseurl = 'http://ly75dbzixy7hlp663j32xo4dtoiikm6bxb53jvivqkpo6jwppptx3sad.onion'

def get_tor_session():
    session = requests.session()
    session.proxies = {'http': 'socks5h://127.0.0.1:9050',
                       'https': 'socks5h://127.0.0.1:9050'}
    return session

@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000)
def get_page_content(session, link):
    return session.get(link)

session = get_tor_session()

print("Getting ...", baseurl)
req = session.get(baseurl)
print("got past r")
soup = BeautifulSoup(req.content, 'lxml')
print("got past SOUP")
productlist = soup.find_all('div', class_='image')

productlinks = []

for item in productlist:
    for link in item.find_all('a', href=True):
        productlinks.append(link['href'])

print(len(productlinks))

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

Getting ... http://ly75dbzixy7hlp663j32xo4dtoiikm6bxb53jvivqkpo6jwppptx3sad.onion
got past r
got past SOUP
130
Execution time: 9.233784914016724 seconds


In [2]:
start_time = time.time()

productlink = productlinks[:5]

#link='http://ly75dbzixy7hlp663j32xo4dtoiikm6bxb53jvivqkpo6jwppptx3sad.onion/venusmarketplace/product/digital/apple-iphone-14-pro-max-1tb-apple-us-original'
productlist = []
vendorlist = []
V = []
for link in productlinks :
    r = get_page_content(session, link)
    soup = BeautifulSoup(r.content, 'lxml')

    name = soup.find('h1').text.strip()
    price = soup.find('span', class_='price-new').text.strip()
    tds = soup.find_all('td')
    li = soup.find_all('li')

    for i, td in enumerate(tds):
        if td.text == 'Vendor Name':
            vname = tds[i + 1].text
        if td.text == 'Total Sales':
            tsales = tds[i + 1].text
        if td.text == 'Member Since':
            doj = tds[i + 1].text

    for i, li in enumerate(li):
        if 'Product:' in li.text :
            cat = li.text.split(': ')[1]

    product = {
        'Product Name': name,
        'Price' : price,
        'Vendor Name' : vname,
        'Category' : cat
    }
    print(product)

    productlist.append(product)
    print('Saving: ', product['Product Name'])

    Vendors = {
        'Vendor Name' : vname,
        'DOJ' : doj,
        'Total Sales' : tsales
    }

    if vname not in V:
        V.append(vname)
        vendorlist.append(Vendors)
        print('Saving: ', Vendors['Vendor Name'])


df1 = pd.DataFrame(productlist)
df2 = pd.DataFrame(vendorlist)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print(df1.head())
print('\n')
print(df2.head())

df1.to_csv('df1F.csv', index=False)  
df1.to_csv('df1.csv') 
df2.to_csv('df2F.csv', index=False)
df2.to_csv('df2.csv')

df1.to_excel('df1F.xlsx', index=False)
df1.to_excel('df1.xlsx' )
df2.to_excel('df2F.xlsx', index=False)
df2.to_excel('df2.xlsx' )

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

{'Product Name': '3 Gram Cocaine Powder High Quality 100% Pure (Best)', 'Price': '$120.00', 'Vendor Name': 'FlexxCocaine', 'Category': 'Cocaine'}
Saving:  3 Gram Cocaine Powder High Quality 100% Pure (Best)
Saving:  FlexxCocaine
{'Product Name': '10 Gram Cocaine Powder High Quality 100% Pure (Best)', 'Price': '$320.00', 'Vendor Name': 'FlexxCocaine', 'Category': 'Cocaine'}
Saving:  10 Gram Cocaine Powder High Quality 100% Pure (Best)
{'Product Name': '20 Gram Cocaine Powder High Quality 100% Pure (Best)', 'Price': '$560.00', 'Vendor Name': 'FlexxCocaine', 'Category': 'Cocaine'}
Saving:  20 Gram Cocaine Powder High Quality 100% Pure (Best)
{'Product Name': '5 Gram Fishscale Cocaine', 'Price': '$150.00', 'Vendor Name': 'hardcoke', 'Category': 'Cocaine'}
Saving:  5 Gram Fishscale Cocaine
Saving:  hardcoke
{'Product Name': '3 Gram Ultraclean 100% Pure Cocaine', 'Price': '$100.00', 'Vendor Name': 'DAGO37', 'Category': 'Cocaine'}
Saving:  3 Gram Ultraclean 100% Pure Cocaine
Saving:  DAGO37
{